## Test per progetto AI

E' necessario caricare i file roads_small.json e nodes_small.json

In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import math
import networkx as nx
import random

from datetime import datetime
from datetime import timedelta

In [2]:
nodes = json.load(open('../datasets/new_nodes.json', 'r'))
roads = json.load(open('../datasets/new_roads.json', 'r'))
print("Num nodes", len(nodes))
print("Num roads", len(roads))

Num nodes 2239
Num roads 10138


In [ ]:
np.random.seed(0) # fix the random seed for np

Create two maps as lookup tables for the nodes

In [3]:
#create a lookup table for point coordinates
node_coords = {point['id']:tuple(point['coordinates']) for point in nodes} #dictonary initialization with comprehension
coords_node = {tuple(point['coordinates']):point['id'] for point in nodes} #tuples are hashable-> can be key to a dictionary

In [4]:
print(node_coords[roads[11]['p1']]) #retrieve coordinates from node ID

(12.55, 45.59)


A questo punto bisogna generare le query degli utenti:

semplificazione: l'orario non influisce sui tempi di percorrenza -> dipendono solo dalla distanza

Consider each trip as starting or ending in one of the nodes: the function below serves to approximate the closest node on the map

Regular transfer (not related to pendolarism) is modeled as a gaussian random variable with respect to location ($\mu = 15km, \sigma = 10km$)  

In [5]:
# compute distance between two nodes

def compute_distance(x_coord_1, y_coord_1, x_coord_2, y_coord_2):
  x_coord_1 = x_coord_1 * 111.32
  x_coord_2 = x_coord_2 * 111.32
  y_coord_1 = 40075 * math.cos(x_coord_1)/360 * y_coord_1;
  y_coord_2 = 40075 * math.cos(x_coord_2)/360 * y_coord_2;
  dist = math.sqrt((y_coord_1 - y_coord_2)**2 + (x_coord_1 - x_coord_2)**2)
  return dist

In [6]:
def compute_distance(x1, y1, x2, y2) :
  R = 6371e3 #metres
  phi1 = x1 * math.pi/180 # φ, λ in radians
  phi2 = x2 * math.pi/180
  delta_phi = (x2-x1) * math.pi/180
  delta_lambda = (y2-y1) * math.pi/180

  a = math.sin(delta_phi/2) * math.sin(delta_phi/2) + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2) * math.sin(delta_lambda/2)
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a));

  d = R * c #in metres

  return d

In [7]:
# use of networkx, creation of graph

G = nx.Graph()

G.add_nodes_from(range(len(nodes)))
for road in roads :
  n1 = int(road.get("p1"))
  n2 = int(road.get("p2"))
  x_coord_1 = float(nodes[n1].get('coordinates')[0])
  y_coord_1 = float(nodes[n1].get('coordinates')[1])
  x_coord_2 = float(nodes[n2].get('coordinates')[0])
  y_coord_2 = float(nodes[n2].get('coordinates')[1])
  dist = compute_distance(x_coord_1, y_coord_1, x_coord_2, y_coord_2)

  G.add_edge(n1, n2, weight=dist)

G.get_edge_data(0, 1)



{'weight': 1553.904995103261}

In [8]:
def generate_ratings(n_driver):
  mean, std = 3.5, 0.5
  x = np.random.normal(mean, std, size=(n_driver, 1))
  ratings = np.empty(shape = (n_driver, 1))
  for i in range(n_driver):
    y = math.ceil(x[i])
    if (y - x[i] > 0.5) :
      ratings[i] = y - 0.5
    else :
      ratings[i] = y
      
  return ratings


In [9]:
# pool of end points 
# define a random number of destinations (e.g. from 6 to 8)
num_dest = random.randrange(6, 8, 1)
pool_dest = []
for i in range(num_dest):
  rand = random.randrange(len(node_coords))
  id_dest = list(node_coords.keys())[rand]
  coord_dest = list(node_coords.values())[rand]
  pool_dest.append({'id':id_dest, 'coordinates':coord_dest})

print(pool_dest)

[{'id': 2238, 'coordinates': (12.39, 45.62)}, {'id': 380, 'coordinates': (12.46, 45.46)}, {'id': 393, 'coordinates': (12.02, 45.21)}, {'id': 272, 'coordinates': (12.16, 45.49)}, {'id': 2115, 'coordinates': (12.62, 45.65)}, {'id': 2037, 'coordinates': (11.81, 45.5)}, {'id': 1808, 'coordinates': (11.86, 45.53)}]


Minimum path lenght:
- Drivers: 40 km
- Riders: 10 km

In [10]:
def generate_trips(n, gen):
  # needs to define, for each driver/rider, one random starting point, and peak from the pool of destinations a random end points
  trips_start = []
  trips_dest = []
  for i in range(n):
    # start point
    if (gen == 'driver') :
      while True:
        # start point
        rand_s = random.randrange((len(nodes)))
        id_start = list(node_coords.keys())[rand_s]
        # end point
        rand_d = random.randrange(num_dest)
        dest = pool_dest[rand_d]
        dist = nx.shortest_path_length(G, id_start, dest.get('id'), weight='weight')
        if (dist > 40e3) :
          coord_start = list(node_coords.values())[rand_s]
          break
    else :
        while True:
          # start point
          rand_s = random.randrange((len(nodes)))
          id_start = list(node_coords.keys())[rand_s]
          # end point
          rand_d = random.randrange(num_dest)
          dest = pool_dest[rand_d]
          dist = nx.shortest_path_length(G, id_start, dest.get('id'), weight='weight')
          if (dist > 10e3) :
            coord_start = list(node_coords.values())[rand_s]
            break
     
    
    trips_start.append({'id':id_start, 'coordinates':coord_start})
    trips_dest.append(dest)

  return trips_start, trips_dest


In [11]:
def generate_schedules(n):
  
  #time probability mass: how many requests per 2-hour slot in percentage
  time_prob = {   
                  '00:00': 1e-3,
                  '2:00': 1e-3,
                  '4:00': 1e-3,
                  '6:00': 1e-3,
                  '6:30': 5e-3,
                  '7:00': 0.02,
                  '7:30': 0.1,
                  '8:00': 0.1,
                  '8:30': 0.1,
                  '10:00': 0.01,
                  '10:30': 0.01,
                  '12:00': 0.1,
                  '12:30': 0.1,
                  '13:00': 0.1,
                  '14:00': 0.01,
                  '16:00': 0.01,
                  '18:00': 0.1,
                  '18:30': 0.1,
                  '19:00': 0.1,
                  '19:30': 0.02,
                  '20:00': 5e-3,
                  '21:00': 5e-3,
                  '22:00': 1e-3,
  }

  

  times = []
  hours = list(time_prob.keys())
  prob = list(time_prob.values())
  for i in range(n):
    times.append((random.choices(hours, prob)[0]))

  return times

In [12]:
class Driver:
  def __init__(self, id, rating, hour, start_point, end_point):
    self.id = id
    self.rating = rating
    self.hour = hour
    self.start_point = start_point
    self.end_point = end_point

  def __str__(self):
    return "Driver ( id = " + str(self.id) + ", rating = " + str(self.rating) + ", hour = " + self.hour + ", start = " + str(self.start_point) + ", dest = " + str(self.end_point) + ")"

class Rider:
  def __init__(self, id, rating, hour, start_point, end_point):
    self.id = id
    self.rating = rating
    self.hour = hour
    self.start_point = start_point
    self.end_point = end_point

  def __str__(self):
    date_format_str = '%H:%M'
    return "Rider ( id = " + str(self.id) + ", rating = " + str(self.rating) + ", hour = " + self.hour + ", start = " + str(self.start_point) + ", dest = " + str(self.end_point) + ")"

In [13]:
def generate_data():
  # 1- generate n_drivers and n_riders
  # range: start, end, stepsize
  n_drivers = random.randrange(40, 50, 1)
  n_riders = random.randrange(150, 250, 15)
  print("N_drivers: ", n_drivers)
  print("N_riders: ", n_riders)
  # 2 - generate start point and end point for drivers and riders
  trips_start_drivers, trips_dest_drivers = generate_trips(n_drivers, gen='driver')
  trips_start_riders, trips_dest_riders = generate_trips(n_riders, gen='rider')
  # 3 - generate schedules
  drivers_schedule = generate_schedules(n_drivers)
  riders_schedule = generate_schedules(n_riders)
  # 4- generate ratings for drivers and riders
  ratings_d = generate_ratings(n_drivers)
  ratings_r = generate_ratings(n_riders)
  # 5 - parse in a list of objects
  riders = []
  drivers = []
  for i in range(n_riders):
    riders.append( Rider(i, ratings_r[i][0], riders_schedule[i], trips_start_riders[i], trips_dest_riders[i]))

  for i in range(n_drivers):
    drivers.append( Driver(i, ratings_d[i][0], drivers_schedule[i], trips_start_drivers[i], trips_dest_drivers[i]))

  return drivers, riders



In [14]:
drivers, riders = generate_data()

for i in range(10) :
  print(drivers[i])

print("\n\n***********************************************************************\n\n")

for i in range(10) :
  print(riders[i])


N_drivers:  46
N_riders:  195
Driver ( id = 0, rating = 3.5, hour = 8:30, start = {'id': 2145, 'coordinates': (12.12, 45.54)}, dest = {'id': 393, 'coordinates': (12.02, 45.21)})
Driver ( id = 1, rating = 3.5, hour = 8:00, start = {'id': 1504, 'coordinates': (12.1, 45.46)}, dest = {'id': 380, 'coordinates': (12.46, 45.46)})
Driver ( id = 2, rating = 4.5, hour = 7:30, start = {'id': 1734, 'coordinates': (12.21, 45.52)}, dest = {'id': 2037, 'coordinates': (11.81, 45.5)})
Driver ( id = 3, rating = 2.5, hour = 12:00, start = {'id': 2012, 'coordinates': (12.09, 45.46)}, dest = {'id': 380, 'coordinates': (12.46, 45.46)})
Driver ( id = 4, rating = 3.0, hour = 18:00, start = {'id': 1278, 'coordinates': (12.64, 45.52)}, dest = {'id': 2037, 'coordinates': (11.81, 45.5)})
Driver ( id = 5, rating = 3.5, hour = 19:00, start = {'id': 610, 'coordinates': (12.13, 45.41)}, dest = {'id': 2115, 'coordinates': (12.62, 45.65)})
Driver ( id = 6, rating = 4.0, hour = 18:00, start = {'id': 1730, 'coordinates':

In [15]:
# use of Dijkstra for shortest path

drivers_path = []
riders_path = []

# 50 km/h
avg_vel = 50
date_format_str = '%H:%M'

for driver in drivers:
  spath = nx.dijkstra_path(G, driver.start_point.get('id'), driver.end_point.get('id'))
  dpath = [{'node' : spath[0], 'hour': driver.hour}]
  dist = 0


  for i in range(1, len(spath)):
    # retrieve distance and convert into km
    
    # distance in km
    dist += (G.get_edge_data(spath[i-1], spath[i]).get('weight')) / 1e3
    time = dist / avg_vel
    time_1 = datetime.strptime(driver.hour, date_format_str)
    time_2 = time_1 + timedelta(hours=time)
    str_time = time_2.strftime(date_format_str)

    dpath.append({'node': spath[i], 'hour':str_time})


  total_dist = nx.shortest_path_length(G, driver.start_point.get('id'), driver.end_point.get('id'), weight='weight')

  drivers_path.append({'id_driver': driver.id, 'shortest_path' : dpath, 'arrival_time': str_time})
  
  #print("Driver #" + str(driver.id) + ": start at " + driver.hour + " , end at " + dpath[len(dpath)-1].get('hour'))
  #print("    -> total dist km: ", total_dist/1e3)


for rider in riders:
  spath = nx.dijkstra_path(G, rider.start_point.get('id'), rider.end_point.get('id'))
  riders_path.append({'id_rider': rider.id, 'shortest_path' : spath})

print(drivers_path[0])
#print(riders_path)

{'id_driver': 0, 'shortest_path': [{'node': 2145, 'hour': '8:30'}, {'node': 2144, 'hour': '08:31'}, {'node': 2146, 'hour': '08:32'}, {'node': 2096, 'hour': '08:33'}, {'node': 1179, 'hour': '08:35'}, {'node': 1178, 'hour': '08:37'}, {'node': 1177, 'hour': '08:38'}, {'node': 1650, 'hour': '08:40'}, {'node': 1670, 'hour': '08:42'}, {'node': 1504, 'hour': '08:43'}, {'node': 787, 'hour': '08:45'}, {'node': 226, 'hour': '08:46'}, {'node': 135, 'hour': '08:49'}, {'node': 133, 'hour': '08:50'}, {'node': 134, 'hour': '08:51'}, {'node': 1048, 'hour': '08:53'}, {'node': 760, 'hour': '08:54'}, {'node': 757, 'hour': '08:56'}, {'node': 1964, 'hour': '08:57'}, {'node': 751, 'hour': '08:59'}, {'node': 1886, 'hour': '09:00'}, {'node': 922, 'hour': '09:02'}, {'node': 1026, 'hour': '09:03'}, {'node': 971, 'hour': '09:05'}, {'node': 976, 'hour': '09:07'}, {'node': 138, 'hour': '09:08'}, {'node': 136, 'hour': '09:09'}, {'node': 1832, 'hour': '09:11'}, {'node': 1612, 'hour': '09:12'}, {'node': 1839, 'hour':

In [16]:
def min_distance (shortest_path_driver, rider) :
  min_distance = float('inf')
  xr = rider.start_point.get('coordinates')[0]
  yr = rider.start_point.get('coordinates')[1]
  for node in shortest_path_driver :
    id = node.get('node')
    x1 = nodes[id].get('coordinates')[0]
    x2 = nodes[id].get('coordinates')[1]
    dist = compute_distance(x1, x2, xr, yr)
    if dist < min_distance :
      min_distance = dist

  return min_distance


In [17]:
# find matching: since we assumed drivers and riders having same dest
# first: filter the total of drivers and riders

drivers_match = [{} for _ in range(len(drivers))]

# couple drivers and riders with same dest and similar hours

for driver in drivers :
  drivers_match[driver.id] = {'id_driver' : driver.id, 'list_of_riders' : []}
  for rider in riders :
    dhour_start = datetime.strptime(driver.hour, date_format_str)
    dhour_end = datetime.strptime(drivers_path[driver.id].get('arrival_time'), date_format_str)
    rhour = datetime.strptime(rider.hour, date_format_str)

    #print("min distance: " , min_distance(drivers_path[driver.id].get('shortest_path'), rider))

    # filter per
    # 1- same dest
    # 2- simil hour
    # 3- minimum distance from rider and shortest path <= 30 km
    if driver.end_point.get('id') == rider.end_point.get('id') and ((dhour_start - timedelta(minutes=30)) <= rhour <= (dhour_end + timedelta(minutes=30)))  :
      drivers_match[driver.id].get('list_of_riders').append(rider.id)
  



In [18]:
def preference_list_drivers(drivers_match) :

  preferences = [{} for _ in range(len(drivers))]

  for driver in drivers :
    pref_dict = {}
    for r in drivers_match[driver.id].get('list_of_riders') :
      rider = riders[r]     
      min_dist = min_distance(drivers_path[driver.id].get('shortest_path'), rider)
      if min_dist <= 35e3 :
        # 0.4 * rating + 0.6 * distance in kms
        measure =  0.4*rider.rating + 0.6*(min_dist/1e3)
        pref_dict[rider.id] = measure

    # sort list
    preferences[driver.id] = {'id_driver' : driver.id, 'list_of_riders' : list(dict(sorted(pref_dict.items(), key=lambda item: item[1])))}

    
    
  return preferences



In [19]:
def preference_list_riders(list_of_riders) :

  preferences = [{} for _ in range(len(riders))]

  pref = [ {} for _ in range(len(riders))]

  # build the lists

  for driver in drivers:
    for r in drivers_match[driver.id].get('list_of_riders') :
      rider = riders[r]
      pref[rider.id][driver.id] = driver.rating
      

  #sort the lists

  for i in range(len(riders)):
    preferences[i] = {'id_rider' : i, 'list_of_drivers' : list(dict(sorted(pref[i].items(), key=lambda item: item[1])))} 
  
  return preferences

# Start of main algorithm

In [20]:
# 1- get path from Steiner tree (first option: compute shortest path)

# 2 - compute preference list of drivers

#print(drivers_match[0].get('list_of_riders'))

print(preference_list_drivers(drivers_match))

print("\n************************************\n")

# 3 - compute preference list of riders

print(preference_list_riders(drivers_match))

[{'id_driver': 0, 'list_of_riders': [128, 140]}, {'id_driver': 1, 'list_of_riders': [82, 142, 159, 97, 23, 192]}, {'id_driver': 2, 'list_of_riders': [166, 164, 89, 77, 28, 86, 52, 106, 57, 84]}, {'id_driver': 3, 'list_of_riders': [155, 147, 112, 90, 139, 186, 181, 99]}, {'id_driver': 4, 'list_of_riders': [25, 165, 153, 41, 167, 174, 119]}, {'id_driver': 5, 'list_of_riders': [183, 146, 156, 93, 180, 1, 94]}, {'id_driver': 6, 'list_of_riders': [49, 123, 4, 189, 178]}, {'id_driver': 7, 'list_of_riders': [97, 82, 192, 142, 23, 159]}, {'id_driver': 8, 'list_of_riders': [147, 139, 99, 90, 181, 51]}, {'id_driver': 9, 'list_of_riders': [179, 6, 184, 32, 191, 152, 8]}, {'id_driver': 10, 'list_of_riders': [113, 124, 76, 54, 38, 7, 145, 193]}, {'id_driver': 11, 'list_of_riders': [20, 120, 172, 30, 68, 10]}, {'id_driver': 12, 'list_of_riders': [147, 139, 90, 99, 181]}, {'id_driver': 13, 'list_of_riders': [122, 36, 61, 37, 92, 73, 5, 151, 2, 55, 62, 105, 163, 24, 102, 43]}, {'id_driver': 14, 'list_

In [21]:
# create file input.dat (obsolete (?))
f = open("input.dat", "w")
content = "DIMENSION : " + str(len(nodes)) + "\n"
content += "DRIVERS : " + str(len(drivers)) + "\n"
content += "RIDERS : " + str(len(riders)) + "\n"
content += "NODE_COORD_SECTION\n"
for node in nodes :
  content += str(node.get("id")) + " " + str(node.get("coordinates")[0]) + " " + str(node.get("coordinates")[1]) + "\n"
content += "ROADS_SECTION\n"
for road in roads :
  content += str(road.get("p1")) + " " + str(road.get("p2")) + "\n"
content += "DRIVERS_STARTING_COORD\n"
for driver in drivers:
  content += str(driver.start_point.get("id")) + " " + str(driver.start_point.get("coordinates")[0]) + " " + str(driver.start_point.get("coordinates")[1]) + "\n"
content += "DRIVERS_DESTINATION_COORD\n"
for driver in drivers:
  content += str(driver.end_point.get("id")) + " " + str(driver.end_point.get("coordinates")[0]) + " " + str(driver.end_point.get("coordinates")[1]) + "\n"
content += "RIDERS_STARTING_COORD\n"
for rider in riders:
  content += str(rider.start_point.get("id")) + " " + str(rider.start_point.get("coordinates")[0]) + " " + str(rider.start_point.get("coordinates")[1]) + "\n"
content += "RIDERS_DESTINATION_COORD\n"
for rider in riders:
  content += str(rider.end_point.get("id")) + " " + str(rider.end_point.get("coordinates")[0]) + " " + str(rider.end_point.get("coordinates")[1]) + "\n"
content += "EOF"
f.write(content)
f.close()

## Stable matching module

Stable matching is modeled after the Hospital-Resident problem, where drivers are modeled as hospitals (their capacity being the number of free seats in the car) and riders are modeled as residents.

We use the open source ```matching``` library, originally released under MIT license by Cardiff University researcher Henry Wilde. https://github.com/daffidwilde/matching

Make sure to install ```matching```: (https://matching.readthedocs.io/en/latest/tutorials/installation.html)

```matching``` is also available through **Conda forge**

In [31]:
from matching.games import HospitalResident

d = {dic["id_driver"]:dic["list_of_riders"] for dic in preference_list_drivers(drivers_match)} #drivers
r = {dic["id_rider"]:dic["list_of_drivers"] for dic in preference_list_riders(drivers_match)} #riders
capacities = {driver_id:4 for driver_id in d} #for the sake of simplicity, every driver is able/willing to transport up to 4 people

game = HospitalResident.create_from_dictionaries(
r, d, capacities
)

game.solve(optimal="hospital") #driver-optimal

TypeError: unhashable type: 'dict'